<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

# Ähnlichkeitsmaße bei Wortformen

Ähnlichkeitsmaße bei Wortformen spielen eine wichtige Rolle bei vielen Verfahren der NLP, insbesondere bei der Rechtschreibkorrektur. 

## Longest Common Substring

Eine simple aber etwas naive Methode nimmt die (gewichtete) Länge des längsten gemeinsamen Substrings als Indikator für die Ähnlichkeit zweier Wörter. 


In [1]:
# Code from https://www.geeksforgeeks.org/sequencematcher-in-python-for-longest-common-substring/
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          return(str1[match.a: match.a + match.size]) 
     else: 
          return("")
    
def lcs_distance(w1,w2) :
    return len(longestSubstring(w1,w2)) / len(w1 + w2)
    
lcs_distance("execution","intention")

0.2222222222222222

Das Problem ist hier natürlich, dass die Unterschiede zwischen zwei Wörtern so verteilt sein können, dass dieses Ähnlichkeitsmaß bei diesen Wörtern trotz der offensichtlichen Ähnlichkeit verhältnismäßig gering ausfällt.

Beispiel:
- *Hunden*
- *Hündin*

In [6]:
lcs_distance("Hunden","Hündin")

0.16666666666666666

## Substring-Überlappung

Ein anderes Ähnlichkeitsmaß betrachtet die Überschneidung der Mengen der Substrings anhand der **Jaccard-Distanz**:

$$jaccard(A,B) = \frac{|A \cap B|}{|A \cup B|}$$

Hier wären $A$ und $B$ die Mengen der Substrings zweier Wortformen.

In [7]:
w1 = "Hunden"
w2 = "Hündin"

w3 = "execution"
w4 = "intention"

def substrings (string) :
    return set([string[i: j] for i in range(len(string)) 
                for j in range(i + 1, len(string) + 1) 
                #if len(string[i: j]) > 1 and len(string[i: j]) < 3 
               ]) 

# Jaccard-Distanz
def jd (setA,setB) :
    return len(setA & setB) / len(setA | setB)

print("{}, {}: {}".format(w1,w2,jd(substrings(w1),substrings(w2))))
print("{}, {}: {}".format(w3,w4,jd(substrings(w3),substrings(w4))))

Hunden, Hündin: 0.1111111111111111
execution, intention: 0.1506849315068493


## Edit Distance (aka Levenshtein Distance)

Die Ähnlichkeit von Wortformen wird standardmäßig mit der **Edit Distance** (Editierdistanz, [Levenshtein Distance](https://de.wikipedia.org/wiki/Levenshtein-Distanz) quantifiziert.

Die Idee ist, dass es eine kleine Menge von Operationen gibt, nämlich {insert, delete, substitute, keep}, die auf Buchstaben angewandt werden können, um ein Wort A in ein Wort B umzuwandeln. 

Das folgende Beispiel (aus Jarafsky & Martin 2019) stellt eine solche Konversion von *intention* nach *execution* dar:

| i            | n            | t            | e            | *            | n            | t            | i            | o            | n            |
|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|
| $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ | $\downarrow$ |
| *            | **e**        | **x**        | **e**        | **c**        | **u**        | **t**        | **i**        | **o**        | **n**        |
| D            | S            | S            |              | I            | S            |              |              |              |              |

Die Konversionsoperationen können dabei unterschiedlich gewichtet werden:
- D(elete): 1
- S(substitute) : 2
- I(nsert): 1
- (keep): 0

Die Operation S ist deshalb "teurer" als D und I, weil hier quasi implizit D und I durchgeführt werden.

Aus den angewandten Operationen und deren Gewicht ergibt sich dann, dass die Edit Distance zwischen *intention* und *execution* 8 beträgt.

## Minimierung der Edit Distance 

(im Wesentlichen übernommen aus Jurafsky & Martin 2019)

Nun gibt es für zwei Wortformen immer trivialerweise unendlich viele mögliche Werte als Edit Distance -- wir können ja beliebig Buchstaben hinzufügen und löschen. Aus diesen möglichen Distanzen interessiert uns aber eigentlich nur die *minimale* Distanz, die sogenannte Minimal Edit Distance oder $D_{Lev}$.

Zur Berechnung von $D_{Lev}$ greift man auf die Methode der Dynamischen Programmierung zurück. D.h. man merkt sich Zwischenergebnisse und berechnet nicht für jeden Edit Path alles neu.

Dafür wird eine Matrix/Tabelle erstellt, so dass die Buchstaben des einen Worts die Zeilen definieren und die Buchstaben des anderen Worts die Spalten. (Es spielt keine Rolle, welche Wortform in den Spalten und welche Wortform in den Zeilen steht.) 

Nehmen wir im Folgenden an, dass $w_1 =$ *execution* und $w_2 =$ *intention*. Das jeweils erste Zeichen ist das leere Zeichen \# und die Tabelle wird in der Zelle $[1,1]$ mit $0$ initialisiert.  

|       | \# | e | x | e | c | u | t | i | o | n |
|-------|----|---|---|---|---|---|---|---|---|---|
| \#    |  $0$ |   |   |   |   |   |   |   |   |   |
| **i** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |
| **t** |    |   |   |   |   |   |   |   |   |   |
| **e** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |
| **t** |    |   |   |   |   |   |   |   |   |   |
| **i** |    |   |   |   |   |   |   |   |   |   |
| **o** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |

Im Anschluss wird die Tabelle von $[1,1]$ bis $[|w_1|+1,|w_2|+1]$ anhand der rekursiven Funktion $D$ ausgefüllt:

$$\begin{array}{ll}
D(x,y) = min ( & D(x-1,y)+1,\\ 
& D(x,y-1)+1,\\
& D(x-1,y-1)+2,\\
& D(x-1,y-1) ~\mathit{iff}~ w_1[x]=x_2[y])
\end{array}$$

Dabei soll gelten: 

$$D(0,y) = |w_2|+1 \text{ und } D(x,0) = |w_1|+1$$ 

Wir erhalten schließlich die folgende vollständig ausgefüllte Tabelle:

|       | \#  | e   | x   | e    | c    | u    | t    | i    | o    | n    |
|-------|-----|-----|-----|------|------|------|------|------|------|------|
| \#    | $0$ | $1$ | $2$ | $3$  | $4$  | $5$  | $6$  | $7$  | $8$  | $9$  |
| **i** | $1$ | $2$ | $3$ | $4$  | $5$  | $6$  | $7$  | $6$  | $7$  | $8$  |
| **n** | $2$ | $3$ | $4$ | $5$  | $6$  | $7$  | $8$  | $7$  | $8$  | $7$  |
| **t** | $3$ | $4$ | $5$ | $6$  | $7$  | $8$  | $7$  | $8$  | $9$  | $8$  |
| **e** | $4$ | $3$ | $4$ | $5$  | $6$  | $7$  | $8$  | $9$  | $10$ | $9$  |
| **n** | $5$ | $4$ | $5$ | $6$  | $7$  | $8$  | $9$  | $10$ | $11$ | $10$ |
| **t** | $6$ | $5$ | $6$ | $7$  | $8$  | $9$  | $8$  | $9$  | $10$ | $11$ |
| **i** | $7$ | $6$ | $7$ | $8$  | $9$  | $10$ | $9$  | $8$  | $9$  | $10$ |
| **o** | $8$ | $7$ | $8$ | $9$  | $10$ | $11$ | $10$ | $9$  | $8$  | $9$  |
| **n** | $9$ | $8$ | $9$ | $10$ | $11$ | $12$ | $11$ | $10$ | $9$  | $8$  |

$D_{Lev}(w_1,w_2)$ ist dann der Inhalt der Zelle $[|w_1|+1,|w_2|+1]$, also $8$.


## <span style="color:red">Aufgaben I</span>

<span style="color:red">A1:</span> Implementieren Sie $D_{Lev}$ in Python ohne NLTK mittels einer Funktion `d_lev(w1,w2)` und führen Sie den darunter stehenden Test aus!

In [46]:
# im nachnihein ein iterativen ansatz versucht...
def d_lev(str1, str2):
    m = len(str1)
    n = len(str2)
    lensum = float(m + n)
    d = []           
    for i in range(m+1):
        d.append([i])        
    del d[0][0]    
    for j in range(n+1):
        d[0].append(j)       
    for j in range(1,n+1):
        for i in range(1,m+1):
            if str1[i-1] == str2[j-1]:
                d[i].insert(j,d[i-1][j-1])           
            else:
                minimum = min(d[i-1][j]+1, d[i][j-1]+1, d[i-1][j-1]+2)         
                d[i].insert(j, minimum)
    ldist = d[-1][-1]
    return ldist

In [7]:
# Lösung A1
# D(x,y)=min(D(x−1,y)+1,
# D(x,y−1)+1,
# D(x−1,y−1)+2,
# D(x−1,y−1) iff w1[x]=x2[y]) 

memo = {}
def d_lev(w1, w2):
    if w1 == "":
        return len(w2)
    if w2 == "":
        return len(w1)
    cost = 0 if w1[-1] == w2[-1] else 1
       
    i1 = (w1[:-1], w2)             # D(x−1,y)+1
    if not i1 in memo:
        memo[i1] = d_lev(*i1)    
    i2 = (w1, w2[:-1])             # D(x,y−1)+1,
    if not i2 in memo:
        memo[i2] = d_lev(*i2)    
    i3 = (w1[:-1], w2[:-1])        # D(x−1,y−1)+2,
    if not i3 in memo:
        memo[i3] = d_lev(*i3)      # D(x−1,y−1) iff w1[x]=x2[y]
    res = min([memo[i1]+1, 
               memo[i2]+1, 
               memo[i3]+cost])     # min(...)
    
    return res                     # D(x,y)

In [47]:
# Tests für d_lev()

[d_lev(i,j) for i,j in 
 [("intention","intention"),
  ("execution","intention"),
  ("inetntion","intention")]]

[0, 8, 2]

## Edit Distance in NLTK

NLTK stellt die entsprechende Funktion `edit_distance(s1,s2)` im Modul [`metrics`](https://www.nltk.org/api/nltk.metrics.html#module-nltk.metrics.distance) zur Verfügung.


In [48]:
from nltk.metrics import edit_distance

[edit_distance(i,j) for i,j in 
 [("intention","intention"),
  ("execution","intention"),
  ("inetntion","intention")]]

[0, 5, 2]

## <span style="color:red">Aufgaben II</span>

<span style="color:red">A2:</span> Ändern Sie den Aufruf von `edit_distance()` so, dass dieselben Werte wie bei `d_lev()` ausgegeben werden!

In [49]:
# Lösung A2
# hmmm meine lösung gibt bereits die selben werte aus :).
# Lösung A1
# D(x,y)=min(D(x−1,y)+1,
# D(x,y−1)+1,
# D(x−1,y−1)+2,
# D(x−1,y−1) iff w1[x]=x2[y]) 

memo = {}
def d_lev(w1, w2):
    if w1 == "":
        return len(w2)
    if w2 == "":
        return len(w1)
    cost = 0 if w1[-1] == w2[-1] else 1
       
    i1 = (w1[:-1], w2)             # D(x−1,y)+1
    if not i1 in memo:
        memo[i1] = d_lev(*i1)    
    i2 = (w1, w2[:-1])             # D(x,y−1)+1,
    if not i2 in memo:
        memo[i2] = d_lev(*i2)    
    i3 = (w1[:-1], w2[:-1])        # D(x−1,y−1)+2,
    if not i3 in memo:
        memo[i3] = d_lev(*i3)      # D(x−1,y−1) iff w1[x]=x2[y]
    res = min([memo[i1]+1, 
               memo[i2]+1, 
               memo[i3]+cost])     # min(...)
    
    return res                     # D(x,y)

# Rechtschreibkorrektur mit Hilfe der Edit Distance

Mit Hilfe der Edit Distance ist es recht einfach, ein Programm zu schreiben, das Vorschläge zur Rechtschreibkorrektur machen kann. Man braucht dafür im Grunde nur noch eine Liste mit Worten, die als Korrektur vorgeschlagen werden sollen. Im Prinzip funktionieren so auch aktuelle Korrekturhilfen wie [Aspell](http://aspell.net/man-html/Aspell-Suggestion-Strategy.html#Aspell-Suggestion-Strategy) oder Hunspell. Wir werden im Folgenden sehen, wie das für das Englische umgesetzt werden kann.

Die Wortliste können wir einfach aus den Worten im Brown Corpus generieren. Außerdem nutzen wir eine Funktion `propose_word_corrections()`, die für einen Eingabestring diejenigen Worte aus einer Wortliste ausgibt, die maximal eine bestimmte Edit Distance (`edt`) zum Eingabestring haben. Als Funktion zur Ermittlung der Edit Distance dient uns diesmal `edit_distance()` aus NLTK.

In [15]:
from nltk.corpus import brown
from nltk.metrics import edit_distance

brownWords = set(brown.words())

def propose_word_corrections (string,edt,lexicon) :
    return [word for word in lexicon if edit_distance(word,string) <= edt]

Mit `propose_word_corrections()` erhalten wir relativ zügig sinnvolle Vorschläge zur Rechtschreibkorrektur.

In [17]:
propose_word_corrections("intention",2,brownWords)

['intentions',
 'intention',
 'infection',
 'intuition',
 'intentioned',
 'invention',
 'Attention',
 'ingestion',
 'intonation',
 'inventions',
 'intentional',
 'inception',
 'attention',
 'contention',
 'retention',
 'Detention',
 'detention',
 'injection',
 'insertion']

Was nun noch stört, ist die Edit Distance Threshold `edt`. Wie könnte die automatisch gesetzt werden?

## <span style="color:red">Aufgaben III</span>

<span style="color:red">A3:</span> Implementieren Sie eine Funktion `lazy_propose_word_corrections(string,lexicon)`, die `edt` automatisch (und möglichst sinnvoll) anhand der Eingabe und der Ausgabe festlegt. Führen Sie damit den anschließenden Test aus.

In [28]:
# Lösung A3
brownWords = set(brown.words())

def lazy_propose_word_corrections(string,lexicon):
    dist_list = []
    for word in lexicon:
        dist_list.append(edit_distance(word, string))
    
    edt = min(dist_list) # habe hier auch mit min(...)+1 gearbeitet, wäre auch sinnvoll um mehr vorschläge zu bekommen 
    print("the edt is: = " + str(edt))
    return [word for word in lexicon if edit_distance(word,string) <= edt]

In [29]:
# Test für lazy_propose_word_corrections()
lazy_propose_word_corrections("inetntion",brownWords)

the edt is: = 2


['intention', 'invention']

## Probleme und Verbesserungsmöglichkeiten

### Umfang der Wortliste

Die Korrekturvorschläge werden auf Grundlage einer Wortliste gemacht. Diese Wortliste sollte im Idealfall alle korrekt geschriebenen Worte einer Sprache enthalten. Oben haben wir dafür das Brown Corpus verwendet und für das Englische war das bereits eine gute Approximation. 

Für eine Sprache wie das Deutsche gestaltet sich das aber schwieriger wegen der reichaltigeren Flexion und der Möglichkeit, Komposita ohne Leerzeichen zu bilden (*Eiersalatsoßengewürz*). Dadurch ist die Anzahl der Wortformen im Deutschen *wesentlich* höher als im Englischen. 

Heutzutage scheint das kein Problem mehr zu sein, denn man kann einfach ein riesigs Webcorpus nehmen (z.B. enthält [DECOW](https://corporafromtheweb.org/decow16/) mehr als 10 Milliarden Token), in dem Millionen von Wortformen enthalten sind. Das Problem ist aber dann, dass solche Ressourcen auch sehr untypische Wortformen enthalten, was die Korrekturhilfe einerseits zu tolerant macht und andererseits zu sehr vielen nutzlosen Vorschlägen führen kann. Und bestimmte Komposita werden auch dort nicht enthalten sein. 

Ein Lösungsansatz besteht darin, mehrstufig vorzugehen: Die Komposita werden zuerst getrennt und dann überprüft.

Außerdem kann die Häufigkeit eines Wortes im Corpus verwendet werden, um die Liste der Korrekturvorschläge klein und möglichst sinnvoll zu halten. 

### Geschwindigkeit

Der Umfang der Wortliste hat auch einen Einfluss auf die Geschwindigkeit der Rechtschreibkorrektur. Wenn so wie bei der  Funktion `propose_word_corrections()` die Edit Distance zu allen Worten der Wortlist einzeln berechnet wird, wird sich das auf die Geschwindigkeit gerade bei umfangreiche Wortlisten spürbar auswirken. Die Wortliste muss also irgendwie stärker strukturiert werden, um den Suchbereich sinnvoll einzugrenzen. 

Zudem könnte man die Berechnung der Edit Distance früher abbrechen, wenn ein Schwellenwert bekannt ist und aus dem Durchlauf der Tabelle klar wird, dass dieser nicht mehr erreicht oder unterboten werden kann.

### Kontextabhängigkeit

Wortformen wie *dass* und *das* sind nicht an alles Positionen im Satz gleich gut. Ein "intelligenter" Rechtschreibkorrektor würde erkennen, dass *das* an der Stelle einer Konjunktion (*dass*, *weil*, *nachdem*, ...) falsch geschrieben ist. Dafür brauchen wir Informationen zum POS-Tag oder Informationen zum Kontext in einem Satz. 

### Editieroperationen und deren Gewichte

Um die Qualität der Rechtschreibkorrektur zu erhöhen, können weitere Editieroperationen hinzugefügt und deren Gewichte optimiert werden. 

Wir haben z.B. oben schon gesehen, dass Subsitution mal mit 1 und mal mit 2 gewichtet wird. Außerdem könnte man sich überlegen, die Gewichtung abhängig von der relativen Position auf der Tastatur festzulegen: Wenn wir eine Substitution von zwei Buchstaben wie *i* und *o* haben, dann sollte das geringer gewichtet werden als eine Substitution von *i* und *s*. Leider ist es gar nicht so trivial, die Gewichte relativ zueinander optimal festzulegen. Man wird hier wohl sehr viele Testdurchläufe brauchen (siehe unten) ...

Was die Editieroperationen betrifft, stellt `edit_distance()` zusätzlich noch die **Transposition** zur Verfügung. Diese Operationen fasst im Grunde eine Deletion und eine Insertion zusammen und ist damit der Substitution sehr ähnlich:

- *in**et**ntion* $\Rightarrow$ *in**te**ntion*

Bei `edit_distance()` kann die Transposition mit dem Argument `transpositions=True` aktiviert werden und wird dann mit 1 gewichtet.

In [30]:
print("without transpositions: " + str(edit_distance("inetntion","intention")))
print("with transpositions: "    + str(edit_distance("inetntion","intention",transpositions=True)))

without transpositions: 2
with transpositions: 1


Es sind natürlich noch weitere Editieroperationen denkbar. Z.B. wäre es manchmal hilfreich, ein Leerzeichen einfügen zu können.

- *allright* $\Rightarrow$ *all right*

## Evaluation durch Testdaten

Wir haben bis hierhin gesehen, dass es eine Reihe von Parametern (Distanzmaß, Editieroperationen, Gewichte, Wortliste, ...) gibt, die man bei der Implementierung einer Rechtschreibkorrektur festlegen muss. Die Frage ist, wie man das so machen kann, dass die Zielgrößen optimiert werden, nämlich: 
- niedriger Ressourcenverbrauch: Zeit, Strom, Speicher
- hohe [Präzision](https://en.wikipedia.org/wiki/Precision_and_recall): $P = \frac{tp}{tp+fp}$
- hoher [Recall](https://en.wikipedia.org/wiki/Precision_and_recall): $R = \frac{tp}{tp+fn}$
- hohes [F1-Maß](https://en.wikipedia.org/wiki/F1_score): $F = \frac{2*P*R}{P+R}$

Außerdem müssen wir die Zielgrößen anhand konkreter Daten, dem sogenannten **Testset**, überprüfen. Es reicht nicht zu sagen: "Ich glaube aber, dass diese Parameter besser sind."

Tatsächlich wirft jedoch der Bedarf eines Testsets eine weitere nichttriviale Frage auf: Wie können wir ein Testset so zusammenstellen, dass die Qualität der Rechtschreibkorrektur *realistischerweise* gemessen werden kann. Tatsächlich erfordert das ein nicht unerhebliches Wissen hinsichtlich der Morphophonologie, der Tippergonomie, der typischen Fehler und der Erwartungshaltung der Nutzer ...

Wir werden im Folgenden die Qualität unserer Rechtschreibkorrektoren anhand eines Testsets überprüfen. Als Testset verwenden wir das Testset von Aspell . 

## <span style="color:red">Aufgaben IV</span>

<span style="color:red">A4:</span> Laden Sie das Testset von Aspell (http://aspell.net/test/cur/batch0.tab) herunter und verändern Sie die folgende Funktion `TEST_propose_word_corrections()` so, dass für eine Eingabe `s1` eine möglichst gute Korrekturliste hinsichtlich des F1-Maßes ausgegeben wird. Testen Sie anschließend die generierten Korrekturlisten mit dem darunter stehenden Skript.

In [39]:
# TBD - not finished yet...
def TEST_propose_word_corrections(s1) :
    out = []
    #precision = tp /(tp+fp)
    #recall = tp/(tp +fn)
    #f1 = (2*precision*recall)/(precision + recall)
    
    # Lösung A4
    dist_list = []
    for word in lexicon:
        dist_list.append(edit_distance(word, string))
    
    edt = min(dist_list) # habe hier auch mit min(...)+1 gearbeitet, wäre auch sinnvoll um mehr vorschläge zu bekommen 
    print("the edt is: = " + str(edt))
    
    out = [word for word in lexicon if edit_distance(word,string) <= edt]
    
    return out

In [45]:
%%time
import csv

sumTrueCorrections = 0
sumFalseCorrections = 0
sumProposedCorrections = 0

with open('batch0.tab') as tsvfile:
  reader = csv.reader(tsvfile, delimiter='\t')
  for row in reader:
        proposedCorrections = TEST_propose_word_corrections(row[0])
        sumProposedCorrections += len(proposedCorrections)
        if row[1] in proposedCorrections :
            sumTrueCorrections += 1
            print("TRUE:  " + row[0] + " :: " + row[1] + "\t" + str(len(proposedCorrections)))
        else :
            sumFalseCorrections += 1
            print("FALSE: " + row[0] + " :: " + row[1] + "\t" + str(len(proposedCorrections)))

recallCorrections = sumTrueCorrections / (sumTrueCorrections + sumFalseCorrections)
precisionCorrections = (sumTrueCorrections + sumFalseCorrections)/sumProposedCorrections

print("Recall:    " + str(recallCorrections))   
print("Precision: " + str(precisionCorrections))
print("F1 score:  " + str(2 * precisionCorrections * recallCorrections / (precisionCorrections + recallCorrections)))
    

FileNotFoundError: [Errno 2] No such file or directory: 'batch0.tab'

Man kann die Ergebnisse dann mit denen von Aspell vergleichen: http://aspell.net/test/cur/